In [63]:

# import necessary libraries 
import pandas as pd 
import os 
import glob 
import numpy as np  # Assuming you have NumPy for creating the array

  
  
# use glob to get all the csv files  
# in the folder 
path = "/Users/axellemoortgat/Projects/SimulationP2PEnergy/Data" 

csv_file_path1 = path + "/100_verbruikers_fluvius.csv"

columns_to_keep = ['EAN_ID', 'ReadStartDate', 'ReadStartDateTime', 'VolumeAfname_TH_kWh', 'VolumeInjectie_TH_kWh']

fluvius = pd.read_csv(csv_file_path1, sep=',', usecols=columns_to_keep)

print(fluvius)


         EAN_ID ReadStartDate         ReadStartDateTime  VolumeAfname_TH_kWh  \
0             1    2021-01-01  2021-01-01T00:00:00.000Z                0.076   
1             1    2021-01-01  2021-01-01T00:15:00.000Z                0.048   
2             1    2021-01-01  2021-01-01T00:30:00.000Z                0.038   
3             1    2021-01-01  2021-01-01T00:45:00.000Z                0.317   
4             1    2021-01-01  2021-01-01T01:00:00.000Z                0.570   
...         ...           ...                       ...                  ...   
3503995     100    2021-12-31  2021-12-31T22:45:00.000Z                0.153   
3503996     100    2021-12-31  2021-12-31T23:00:00.000Z                0.122   
3503997     100    2021-12-31  2021-12-31T23:15:00.000Z                0.139   
3503998     100    2021-12-31  2021-12-31T23:30:00.000Z                0.170   
3503999     100    2021-12-31  2021-12-31T23:45:00.000Z                0.155   

         VolumeInjectie_TH_kWh  
0     

## Opslitsing Fluvius data

In [64]:
grouped_data = fluvius.groupby('EAN_ID')

datasets = {}
prosumenten = {}

for group_name, group_data in grouped_data:
    datasets[group_name] = group_data
    total_injection = group_data['VolumeInjectie_TH_kWh'].sum()
    if total_injection > 0:
        prosumenten[group_name] = group_data


## Inlezen Day Ahead data

In [65]:
csv_file_path1 = path + "/Day-ahead Prices_2023.csv"

columns_to_keep = ['MTU (CET/CEST)', 'Day-ahead Price [EUR/MWh]']

day_ahead = pd.read_csv(csv_file_path1, sep=',', usecols=columns_to_keep)

print(day_ahead)


                           MTU (CET/CEST)  Day-ahead Price [EUR/MWh]
0     01.01.2023 00:00 - 01.01.2023 01:00                      -4.39
1     01.01.2023 01:00 - 01.01.2023 02:00                      -1.75
2     01.01.2023 02:00 - 01.01.2023 03:00                      -1.46
3     01.01.2023 03:00 - 01.01.2023 04:00                      -5.27
4     01.01.2023 04:00 - 01.01.2023 05:00                      -4.41
...                                   ...                        ...
8756  31.12.2023 19:00 - 31.12.2023 20:00                      20.26
8757  31.12.2023 20:00 - 31.12.2023 21:00                      19.65
8758  31.12.2023 21:00 - 31.12.2023 22:00                       9.00
8759  31.12.2023 22:00 - 31.12.2023 23:00                      10.68
8760  31.12.2023 23:00 - 01.01.2024 00:00                       3.17

[8761 rows x 2 columns]


## Kwartierwaardes fluvius omzetten naar uurwaardes

In [69]:
prosument = prosumenten[1]
prosument['ReadStartDateTime'] = pd.to_datetime(prosument['ReadStartDateTime'])

# Converteer 'ReadStartDateTime' naar datetime-formaat
prosument['ReadStartDateTime'] = pd.to_datetime(prosument['ReadStartDateTime'], utc=True)

# Verander het jaartal naar 2023 in de kolom 'ReadStartDateTime'
prosument['ReadStartDateTime'] = prosument['ReadStartDateTime'].dt.strftime('%Y-%m-%d %H:%M:%S+00:00').str.replace('2021', '2023')

# Converteer opnieuw naar datetime-objecten
prosument['ReadStartDateTime'] = pd.to_datetime(prosument['ReadStartDateTime'], utc=True)

# Rond 'ReadStartDateTime' af naar het dichtstbijzijnde uur
prosument['Hour'] = prosument['ReadStartDateTime'].dt.floor('H')

# Groepeer op 'Hour', en bereken de som van 'VolumeInjectie_TH_kWh'
hourly_sum = prosument.groupby(['Hour'])['VolumeInjectie_TH_kWh'].sum().reset_index()

# Voeg een nieuwe kolom toe met het gewenste tijdsinterval
hourly_sum['TimeRange'] = hourly_sum['Hour'].dt.strftime('%d.%m.%Y %H:%M') + ' - ' + (hourly_sum['Hour'] + pd.DateOffset(hours=1)).dt.strftime('%d.%m.%Y %H:%M')

# Toon het resultaat
print(hourly_sum[['TimeRange', 'VolumeInjectie_TH_kWh']])

                                TimeRange  VolumeInjectie_TH_kWh
0     01.01.2023 00:00 - 01.01.2023 01:00                    0.0
1     01.01.2023 01:00 - 01.01.2023 02:00                    0.0
2     01.01.2023 02:00 - 01.01.2023 03:00                    0.0
3     01.01.2023 03:00 - 01.01.2023 04:00                    0.0
4     01.01.2023 04:00 - 01.01.2023 05:00                    0.0
...                                   ...                    ...
8755  31.12.2023 19:00 - 31.12.2023 20:00                    0.0
8756  31.12.2023 20:00 - 31.12.2023 21:00                    0.0
8757  31.12.2023 21:00 - 31.12.2023 22:00                    0.0
8758  31.12.2023 22:00 - 31.12.2023 23:00                    0.0
8759  31.12.2023 23:00 - 01.01.2024 00:00                    0.0

[8760 rows x 2 columns]


In [74]:
all_hourly_sums = {}

for ean_id,prosument in prosumenten.items():
    # Converteer 'ReadStartDateTime' naar datetime-formaat
    prosument['ReadStartDateTime'] = pd.to_datetime(prosument['ReadStartDateTime'], utc=True)

    # Verander het jaartal naar 2023 in de kolom 'ReadStartDateTime'
    prosument['ReadStartDateTime'] = prosument['ReadStartDateTime'].dt.strftime('%Y-%m-%d %H:%M:%S+00:00').str.replace('2021', '2023')

    # Converteer opnieuw naar datetime-objecten
    prosument['ReadStartDateTime'] = pd.to_datetime(prosument['ReadStartDateTime'], utc=True)

    # Rond 'ReadStartDateTime' af naar het dichtstbijzijnde uur
    prosument['Hour'] = prosument['ReadStartDateTime'].dt.floor('H')

    # Groepeer op 'Hour', en bereken de som van 'VolumeInjectie_TH_kWh'
    hourly_sum = prosument.groupby(['Hour'])['VolumeInjectie_TH_kWh'].sum().reset_index()

    # Voeg een nieuwe kolom toe met het gewenste tijdsinterval
    hourly_sum['TimeRange'] = hourly_sum['Hour'].dt.strftime('%d.%m.%Y %H:%M') + ' - ' + (hourly_sum['Hour'] + pd.DateOffset(hours=1)).dt.strftime('%d.%m.%Y %H:%M')

    # Voeg de hourly_sum toe aan de grote dictionary met de EAN_ID als sleutel
    all_hourly_sums[prosument['EAN_ID'].iloc[0]] = hourly_sum[['TimeRange', 'VolumeInjectie_TH_kWh']]

# Toon de resultaten voor alle prosumenten
for ean_id, hourly_sum in all_hourly_sums.items():
    print(f"Resultaten voor prosument {ean_id}:")
    print(hourly_sum)
    print("\n")

Resultaten voor prosument 1:
                                TimeRange  VolumeInjectie_TH_kWh
0     01.01.2023 00:00 - 01.01.2023 01:00                    0.0
1     01.01.2023 01:00 - 01.01.2023 02:00                    0.0
2     01.01.2023 02:00 - 01.01.2023 03:00                    0.0
3     01.01.2023 03:00 - 01.01.2023 04:00                    0.0
4     01.01.2023 04:00 - 01.01.2023 05:00                    0.0
...                                   ...                    ...
8755  31.12.2023 19:00 - 31.12.2023 20:00                    0.0
8756  31.12.2023 20:00 - 31.12.2023 21:00                    0.0
8757  31.12.2023 21:00 - 31.12.2023 22:00                    0.0
8758  31.12.2023 22:00 - 31.12.2023 23:00                    0.0
8759  31.12.2023 23:00 - 01.01.2024 00:00                    0.0

[8760 rows x 2 columns]


Resultaten voor prosument 14:
                                TimeRange  VolumeInjectie_TH_kWh
0     01.01.2023 00:00 - 01.01.2023 01:00                    0.0
1   

## Mapping prijzen op fluvius

In [96]:
total_volume_injectie = 0
total_profit = 0
for ean_id, hourly_sum in all_hourly_sums.items():
    print(f"Resultaten voor prosument {ean_id}:")
    
    merged_df = pd.merge(day_ahead, hourly_sum, left_on='MTU (CET/CEST)', right_on='TimeRange', how='inner')
    merged_df['Profit'] = merged_df['Day-ahead Price [EUR/MWh]'] * merged_df['VolumeInjectie_TH_kWh']/1000
    print(merged_df['Profit'].sum())
    total_volume_injectie += merged_df['VolumeInjectie_TH_kWh'].sum()
    total_profit += merged_df['Profit'].sum()
    print("\n")

Resultaten voor prosument 1:
182.31313317605003


Resultaten voor prosument 14:
181.00467040752


Resultaten voor prosument 16:
275.03126589791


Resultaten voor prosument 17:
273.85977909652


Resultaten voor prosument 19:
230.57295152752


Resultaten voor prosument 20:
197.04610301726


Resultaten voor prosument 21:
155.54407734


Resultaten voor prosument 23:
347.90897937164993


Resultaten voor prosument 24:
189.66413127621


Resultaten voor prosument 30:
113.5502259852


Resultaten voor prosument 33:
283.88920844186003


Resultaten voor prosument 34:
97.48265932991


Resultaten voor prosument 36:
224.22777655204


Resultaten voor prosument 37:
173.20875106265


Resultaten voor prosument 40:
523.01995803422


Resultaten voor prosument 41:
347.0181689125


Resultaten voor prosument 43:
384.13737546999994


Resultaten voor prosument 44:
112.50203352622


Resultaten voor prosument 45:
151.64916082485


Resultaten voor prosument 47:
239.48120692613998


Resultaten voor prosument 50:
27

In [97]:
print(total_volume_injectie)
print(total_profit)

156078.928124
12863.45767823279


## Mapping Engieprijzen flow

In [81]:
Injection_price = np.array([0.082,0.09,0.068,0.066,0.05,0.058,0.057,0.07,0.072,0.066,0.069,0.078])

In [98]:
total_profit = 0
for ean_id, hourly_sum in all_hourly_sums.items():
    print(f"Resultaten voor prosument {ean_id}:")
    # Extract the start date from 'TimeRange'
    hourly_sum['StartDate'] = pd.to_datetime(hourly_sum['TimeRange'].str.split(' - ').str[0], format='%d.%m.%Y %H:%M')

    # Add a new column 'Price injectie Engie' based on the month
    hourly_sum['Price injectie Engie'] = hourly_sum['StartDate'].dt.month.map(lambda x: Injection_price[x - 1])
    
    hourly_sum['Profit'] = hourly_sum['VolumeInjectie_TH_kWh'] * hourly_sum['Price injectie Engie']
    total_profit += hourly_sum['Profit'].sum()
    print(hourly_sum['Profit'].sum())
    print("\n")

Resultaten voor prosument 1:
149.091497066


Resultaten voor prosument 14:
147.140274068


Resultaten voor prosument 16:
208.884205074


Resultaten voor prosument 17:
209.49223300000003


Resultaten voor prosument 19:
163.13323330199998


Resultaten voor prosument 20:
134.74508256899998


Resultaten voor prosument 21:
115.31170400000002


Resultaten voor prosument 23:
275.708327184


Resultaten voor prosument 24:
152.42309348


Resultaten voor prosument 30:
87.35859297


Resultaten voor prosument 33:
231.93490608200005


Resultaten voor prosument 34:
79.394189829


Resultaten voor prosument 36:
157.41829255


Resultaten voor prosument 37:
148.18190009


Resultaten voor prosument 40:
427.741921


Resultaten voor prosument 41:
249.599447


Resultaten voor prosument 43:
306.88222099999996


Resultaten voor prosument 44:
99.317364952


Resultaten voor prosument 45:
120.50214033600001


Resultaten voor prosument 47:
187.373987056


Resultaten voor prosument 50:
231.79267111800002


Resultat

In [99]:
print(total_profit)

10014.376038724999
